In [1]:
import tensorflow as tf
import os
import pandas as pd

In [ ]:
# !pip install fitz
!pip install pymupdf


In [2]:
import fitz


In [4]:
# Load Transcript PDF File
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

In [ ]:
!ls

In [ ]:
!unzip earning_call_validation.zip

In [5]:
earning_call_train_path = "Earning Call Transcript - Training Data"

In [6]:
!ls 

aclImdb					   glove.6B.zip
aclImdb_v1.tar.gz			   glove_lstm_model.keras
baseline.ipynb				   glove_vecs.bin
earning_call_train.zip			   glove_word_averaging_model.h5
Earning Call Transcript - Training Data    glove_word_averaging_model.keras
Earning Call Transcript - Validation Data  imdb_full_dataset.csv
earning_call_validation.zip		   infer_financial_sentiments.ipynb
financial_data_kaggle.csv		   run_model_on_our_data.ipynb
financial_labels.csv			   using_glove.ipynb
glove.42B.300d.txt			   vocab_index.json
glove.42B.300d.zip


In [7]:
earning_call_validation_path = "Earning Call Transcript - Validation Data"

In [8]:
pdf_files = os.listdir(earning_call_train_path)

In [9]:
labels_df = pd.read_csv('financial_labels.csv')

In [10]:
def match_pdf_to_label(pdf_filename, labels_df):
    # Remove extension and any known formatting from the PDF filename
    clean_pdf_filename = pdf_filename.lower().replace('.pdf', '').replace('_', ' ')
    # Attempt to match the cleaned PDF filename with entries in the labels dataframe
    for _, row in labels_df.iterrows():
        if row['file_name'].lower() in clean_pdf_filename:
            return row['label']
    return None  # If no match is found, you might want to handle this case

In [11]:
# Dictionary to store matched labels with PDF filenames
matched_labels = {}

In [12]:
# Iterate over PDF files and match them to labels
for pdf_file in pdf_files:
    if pdf_file.endswith('.pdf'):
        matched_label = match_pdf_to_label(pdf_file, labels_df)
        matched_labels[pdf_file] = matched_label

In [ ]:
matched_labels

In [ ]:
def preprocess_text(text):
    # Tokenize and convert to lower case, initial tokenization for removing stop words, punctuation, and normalizing the text.
    tokens = word_tokenize(text.lower())
    
    # Remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    stripped_tokens = [w.translate(table) for w in tokens]
    
    # Remove remaining tokens that are not alphabetic
    words = [word for word in stripped_tokens if word.isalpha()]
    
    # Filter out stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    
    return words

In [ ]:
def clean_transcript(text):
    # Remove URLs in the footer
    text = re.sub(r'https?://\S+', '', text)
    
    # Normalize text to lower case
    text = text.lower()

    # Remove date and time patterns, e.g., "2/7/24, 8:42 AM"
    text = re.sub(r'\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{2} [APM]{2}', '', text)

    # Remove any text that looks like a website, e.g., "seeking alpha"
    text = re.sub(r'seeking alpha', '', text, flags=re.IGNORECASE)

    # Remove headers that might be repeated, e.g., earnings call transcript titles
    text = re.sub(r'earnings call transcript', '', text)

    # Remove numeric patterns that don't add value, e.g., "Q4 2023"
    text = re.sub(r'q[1-4] \d{4}', '', text)

    # Remove specific phrases or patterns that don't contribute to the analysis
    phrases_to_remove = [
        r'\bfeb\. \d{2}, 2024\b',
        r'\b\d{1,2}:\d{2} [apm]{2} et\b',
        r'\btranscripts\b',
        r'\bfollowers\b',
        # Add more patterns as needed
    ]
    for phrase in phrases_to_remove:
        text = re.sub(phrase, '', text)

    # Remove any leading or trailing whitespace
    text = text.strip()

    return text

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string


In [ ]:
import re
# Preprocess texts and store them
preprocessed_texts = {}
for pdf_file in pdf_files:
    if pdf_file.endswith('.pdf'):
        pdf_path = os.path.join(earning_call_train_path, pdf_file)
        text = extract_text_from_pdf(pdf_path)
        # Preprocess the extracted text
        clean_text = clean_transcript(text)
        processed_text = preprocess_text(clean_text)
        # Store the processed text with its matched label
        matched_label = matched_labels.get(pdf_file)
        preprocessed_texts[pdf_file] = (processed_text, matched_label)

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

In [ ]:
preprocessed_texts